# Inteligência Artificial - Trabalho
- Universidade Federal de Santa Catarina
- Departamento de Automação e Sistemas
- Prof. Eric Aislan Antonelo

### Grupo
-

### Opção: 1

#### 1. Implementação:
- Crie as estruturas de dados para guardar os pesos que definem uma arquitetura
de rede neural multi-camadas. Inicialize a rede neural aleatoriamente.
- Implemente o algoritmo da retropropagação para o cálculo do gradiente, a
derivada parcial da função de custo com relação aos pesos da rede.
- Valide o algoritmo do cálculo do gradiente, realizando uma aproximação numérica
do mesmo. Verifique se os cálculos batem um com o outro.
- Dado o gradiente já calculado, implemente o método do descenso do gradiente
para o treinamento da rede neural, ou seja, o processo de ajuste dos pesos.

#### 2. Aplicação:
- Use o código implementado para treinar uma rede neural para realizar a classificação de um padrão de duas dimensões de entrada. Os dados para treinamento
estão disponíveis no arquivo
classification2.txt.
Para plotar a fronteira de decisão da rede treinada, poderá usar o código
disponível no link
https://colab.research.google.com/drive/1XTtZGgpAefbiWejTrEjsnWzS_
XXYdzff?usp=sharing.
- Relate resultados variando pelo menos duas vezes cada um dos hiperparâmetros: o número de camadas; o número de neurônios por camada; taxa de aprendizagem. Use métricas como taxa de classificação (porcentagem de predições
corretas) no conjunto de validação (exemplos não usados no treinamento).
- (opcional) Treine uma rede neural para classificar dígitos a partir de imagens
como entrada para a rede. Use o arquivo
classification3.mat.

#### 3. Entregas:
No relatório a ser entregue, descreva os experimentos e os resultados obtidos.
Grave um video de até 3 minutos, onde você deve explicar o código implementado
de uma forma geral, as dificuldades encontradas, e em especial:
- a parte do código referente ao cálculo do gradiente
- a parte do código referente ao gradient descent
- o gráfico da fronteira de decisão
Entregue o código, PDF do relatório e o arquivo de video pelo Moodle (zipado com
ZIP ou tar.gz).

In [1]:
import pandas as pd
import numpy as np

### Data source

In [2]:
df = pd.read_csv('classification2.txt', header=None)
df.columns = ['column1', 'column2', 'label']
df.info()
print(f'\nLabel Value counts: \n{df.label.value_counts()}')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   column1  118 non-null    float64
 1   column2  118 non-null    float64
 2   label    118 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.9 KB

Label Value counts: 
0    60
1    58
Name: label, dtype: int64


,column1,column2,label
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [25]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### Neural net data structure

In [175]:
class Neuron():
    def __init__(self, input_length):
        self.bias = 1
        self.weights = self.create_random_weights(input_length)
        
    def create_random_weights(self, input_length):
        return np.random.rand(input_length)
        
    def __repr__(self):
        return f'Neuron {self.weights}'
    
class Layer():
    def __init__(self, input_length, n_neurons):
        self.input_length = input_length
        self.n_neurons = n_neurons
        self.create_neurons()
    
    def create_neurons(self):
        """Creates neurons of layer
        """
        self.neurons = list()
        for i in range(self.n_neurons):
            self.neurons.append(
                Neuron(self.input_length)
            )
            
    def as_weight_matrix(self):
        """Represents layer's weights as a matrix, where each column is a neuron.
        """
        return np.array([n.weights for n in self.neurons]).T
    
    def as_bias_matrix(self):
        """Represents layer's biases as a matrix, where each column is a neuron.
        """
        return [n.bias for n in self.neurons]
            
    def forward(self, input_data):
        """Calculates layer output based on input data
        """
        return np.dot(input_data, self.as_weight_matrix()) + self.as_bias_matrix()
    
class ReLU():
    def forward(self, inputs):
        return np.maximum(0, inputs)

In [180]:
layer1 = Layer(input_length=X.shape[1], n_neurons=3)
layer2 = Layer(input_length=3, n_neurons=2)

print(layer1.neurons)

[Neuron [0.75454365 0.07113508], Neuron [0.79581706 0.85613737], Neuron [0.29683669 0.54290816]]


In [181]:
layer1_output = layer1.forward(X)
layer1_output

array([[1.08844645, 1.63971861, 1.39501476],
       [0.97874537, 1.51259706, 1.34433029],
       [0.88798974, 1.42258703, 1.31239121],
       [0.75276946, 1.13151223, 1.16132929],
       [0.64585381, 0.9901987 , 1.10044833],
       [0.61896227, 0.7619967 , 0.95813114],
       [0.70210543, 0.71264729, 0.90049982],
       [0.75552447, 0.59198307, 0.8048295 ],
       [0.98384901, 0.66720915, 0.78549346],
       [1.06297625, 0.6650158 , 0.76016065],
       [1.2505829 , 0.82291504, 0.80761627],
       [1.36236471, 0.97507084, 0.87417567],
       [1.46470187, 1.29998289, 1.05747051],
       [1.54275431, 1.42798417, 1.11828899],
       [1.44716216, 1.85246825, 1.42697448],
       [1.28440635, 1.75503872, 1.41187971],
       [1.16393219, 1.59371511, 1.34190075],
       [1.02287696, 1.66192126, 1.42944527],
       [0.91893293, 1.46093274, 1.32832832],
       [0.68389078, 1.16164451, 1.20198615],
       [0.58567502, 1.02950676, 1.14452771],
       [0.54956288, 0.78586206, 0.99481392],
       [0.

In [182]:
layer2.forward(layer1_output)

array([[2.82184792, 2.70922247],
       [2.67077506, 2.60538375],
       [2.5573289 , 2.53344167],
       [2.26883559, 2.28165067],
       [2.10834057, 2.16440089],
       [1.92134118, 1.95742656],
       [1.92495208, 1.90190693],
       [1.85956698, 1.78428529],
       [2.02988193, 1.82596705],
       [2.06739447, 1.81437493],
       [2.28046902, 1.93714333],
       [2.45162565, 2.06379274],
       [2.74958939, 2.3507334 ],
       [2.88566017, 2.4592109 ],
       [3.16139788, 2.86182288],
       [3.00665204, 2.79175513],
       [2.8242136 , 2.65771148],
       [2.80627782, 2.73760781],
       [2.60183594, 2.565024  ],
       [2.25766206, 2.31774136],
       [2.10845864, 2.20789223],
       [1.90514009, 1.98780693],
       [1.71306869, 1.75338849],
       [1.69152118, 1.64761532],
       [1.9051569 , 1.74663266],
       [2.07484675, 1.79031661],
       [2.37952449, 2.01171301],
       [2.58261669, 2.14482206],
       [2.5302565 , 2.46746038],
       [2.5386046 , 2.55011217],
       [2.